In [1]:
import jax

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-23 07:57:12.087493: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-23 07:57:12.087514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-23 07:57:12.087529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-23 07:57:13.637538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-40GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=20_000, max_seq_len=512)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 19169
[  103    99   125   417   109    17   106    42  2428   126  2235   147
   108    42  4304  4516   165    97   101    99  1181    15    95  3590
    15 13681   236    15  3897  2322    15  3476    96    95   129    17
   101    99   864    98   285    95  1181   107  2655   110   192   239
   103   111    42   263   303   101   138  2221   113   162    17    31
   100    18    33    31   100    18    33    95   154    97  3222  7397
    99   163   664    98   200 16932  1196    98   248  5785    96   972
   637  4205   107  9998   236    95   154    17    31   100    18    33
    31   100    18    33  3770   217   163  2424    42  1429   216    15
    42   146   575   330    17  4069  5911  8326  8743    99  1436   102
   131   175  1153   308    17    95   456    97    95   270    99  1255
    17     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0    

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=64, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=32)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=1e-3, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 97.88batch/s, Loss = 2.0457, AUC = 88.80%] 

TOTAL TIME = 249.16s
BEST AUC = 94.52% AT EPOCH 1


In [5]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=6, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=3)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=1e-3, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 99.68batch/s, Loss = 0.8332, AUC = 90.70%] 

TOTAL TIME = 234.49s
BEST AUC = 92.72% AT EPOCH 5
